In [22]:
import numpy as np

R = 8.31446261815324
t = 298.15
h =  6.6260755455 * 10 **(-34)
speed_light = 2.99792458 * 10 ** (8)
k = 1.380649 * 10 ** (-23)
const_avocado = 6.022141 * 10**(23)
pressure = 101325
mass = 46.0055 

def convert_w_to_nu(w_array):
    return w_array * 100 * speed_light 

def ent_vib(nu_array, T):
    sum_entropy = 0
    kote = k * T
    for nu in nu_array:
        useful_everywhere = -h*nu/kote
        sum_entropy += -useful_everywhere * np.exp(useful_everywhere) / (1 - np.exp(useful_everywhere)) - np.log(1 - np.exp(useful_everywhere)) 
    return sum_entropy * R

def ene_vib(nu_array, T):
    sum_energy = 0
    kote = k * T
    for nu in nu_array:
        useful_everywhere = -h*nu/kote
        sum_energy += -useful_everywhere * np.exp(useful_everywhere) / (1 - np.exp(useful_everywhere))
    return T * R * sum_energy

def ent_rot_nonlin(mom_inert, sigma, T):
    kote = k * T
    q = np.sqrt(np.pi)/sigma * ((8 * np.pi ** 2 * kote)/ h ** 2) ** (3/2) * np.sqrt(mom_inert[0] * mom_inert[1] * mom_inert[2])
    return R * np.log(q) + 3/2 * R

def ene_rot_nonlin(T):
    return R * T * 3 / 2
    
def ent_rot_lin(mom_inert, sigma, T):
    kote = k * T
    q = (8 * np.pi ** 2 * mom_inert * kote)/ sigma / h ** 2
    return R * np.log(q) + R

def ene_rot_lin(T):
    return R * T

def ent_el(g):
    return R * np.log(g)

def ene_trans(T):
    return 3 / 2 * R * T

def ent_trans(T, m, press):
    mass_molecure = m / const_avocado * 10 ** (-3)
    kote = k * T
    q = kote/ press * (np.sqrt(2 * np.pi * mass_molecure * kote)/ h) ** 3
    return 5/2 * R + R * np.log(q)
    
    
    
w_array  = np.array([623.49, 743.27, 1412.65])
nu_array = convert_w_to_nu(w_array)
moment_inert = (1.009173 * 10 **(-46), 5.013162 * 10 **(-46), 6.022335 * 10 **(-46))
sigma = 2


print(ent_el(2))


5.763146321643979


In [45]:
import random
import itertools
import math


R = 8.314

class Cell:
    def __init__(self):
        self.isborrow = False

class Pole:
    def __init__(self, N, coordsparticular):
        self.N = N
        self.__pole = [[Cell() for _ in range(N)] for _ in range(N)]
        self.particular = {}  # Словарь для хранения координат частиц

        for idx, (x, y) in enumerate(coordsparticular):
            self.__pole[y][x].isborrow = True
            self.particular[idx] = (x, y)

    def change(self, coords_before, coords_after):
        x_before, y_before = coords_before
        x_after, y_after = coords_after
        self.__pole[y_before][x_before].isborrow = False
        self.__pole[y_after][x_after].isborrow = True

        # Обновление координат частиц в словаре
        for idx, (x, y) in self.particular.items():
            if (x, y) == coords_before:
                self.particular[idx] = (x_after, y_after)
                print(self.particular)

    def get_pole(self):
        return self.__pole

class MonteCarlo:
    def __init__(self, N, coordsparticular):
        self.pole = Pole(N, coordsparticular)
        self.particular = list(range(len(coordsparticular)))
        self.current_particular = 0  # Переменная для определения, какая частица ходит
        self.__distances = []  # Внутренняя переменная для сохранения расстояний
        self.distance_to_energy = {}  # Словарь расстояний к энергии

    def move(self, temperature):

        N = self.pole.N
        if not self.particular:
            return  # Нет частиц для перемещения

        num_particular = len(self.particular)
        current_particular = self.particular[self.current_particular]

        available_cells = [i for i in range(N * N) if (i % N, i // N) not in self.pole.particular.values()] + [current_particular]

        if not available_cells:
            return  # Нет доступных ячеек для перемещения

        n = random.choice(available_cells) 
        x_before, y_before = self.pole.particular[current_particular]
        x_after = n % N
        y_after = n // N
        coords_before = (x_before, y_before)
        coords_after = (x_after, y_after)
        sum_energy_before = 0
        sum_energy_after = 0
        external_particular = list(self.pole.particular.values())
        external_particular.remove((x_before, y_before))
        for x, y in external_particular:        
            energy_before_before = self.distance_to_energy.get((x - x_before) ** 2 + (y - y_before) ** 2, 0)
            energy_after_after = self.distance_to_energy.get((x - x_after) ** 2 + (y - y_after) ** 2, 0)
            sum_energy_before += energy_before_before
            sum_energy_after += energy_after_after 
            
        energy_after = sum_energy_after
        energy_before = sum_energy_before
        delta_energy = energy_after - energy_before

        random_value = random.random()
        probably_change = 2.71828 ** (-delta_energy / (R * temperature))
        if delta_energy <= 0 :
            print(f"energy_after = {energy_after}, energy_bofore = {energy_before}")
            print(f"different_energy = {delta_energy} <= 0")
            self.pole.change(coords_before, coords_after)
            print("Переход разрешен ")
        elif delta_energy > 0 and random_value  < probably_change:
            print(f"energy_after = {energy_after}, energy_bofore = {energy_before}\n")
            print(f"Переход состоялся random_value = {random_value} < probably_change = {probably_change}")
            self.pole.change(coords_before, coords_after)
        else:
            print(f"energy_after = {energy_after}, energy_bofore = {energy_before}\n")
            print(f"Переход не состоялся random_value = {random_value} > probably_change = {probably_change}")


    def all_distances(self):
        borrowed_cells = [(x, y) for y in range(self.pole.N) for x in range(self.pole.N)]
        distances = set()

        for (x1, y1), (x2, y2) in itertools.combinations(borrowed_cells, 2):
            distance = (x2 - x1) **2 + (y2 - y1) ** 2  # Манхэттенское расстояние
            distances.add(distance)

        self.__distances = list(distances)  # Сохраняем расстояния во внутренней переменной
        return self.__distances

    def determinate_distance(self, energy):
        if len(energy) != len(self.__distances):
            raise ValueError("Количество энергий должно соответствовать количеству расстояний")

        self.distance_to_energy = {self.__distances[i]: energy[i] for i in range(len(self.__distances))}

# Пример использования:
N = 3
coordsparticular = [(1, 1), (2, 2)]
monte_carlo_sim = MonteCarlo(N, coordsparticular)

monte_carlo_sim.all_distances()

monte_carlo_sim.determinate_distance([-1000, -700, -200, 0, 0])
for i in range(20):
    random_value = int(random.random() * 2)
    monte_carlo_sim.current_particular = random_value
    monte_carlo_sim.move(300)
    for row in monte_carlo_sim.pole.get_pole():
        print([ "*" if cell.isborrow else "-" for cell in row])
    
    
    
    





energy_after = -1000, energy_bofore = -700
different_energy = -300 <= 0
{0: (1, 1), 1: (2, 1)}
Переход разрешен 
['-', '-', '-']
['-', '*', '*']
['-', '-', '-']
energy_after = 0, energy_bofore = -1000

Переход не состоялся random_value = 0.781706850608256 > probably_change = 0.6696970130016205
['-', '-', '-']
['-', '*', '*']
['-', '-', '-']
energy_after = -1000, energy_bofore = -1000
different_energy = 0 <= 0
{0: (1, 1), 1: (0, 1)}
Переход разрешен 
['-', '-', '-']
['*', '*', '-']
['-', '-', '-']
energy_after = -1000, energy_bofore = -1000
different_energy = 0 <= 0
{0: (1, 1), 1: (1, 2)}
Переход разрешен 
['-', '-', '-']
['-', '*', '-']
['-', '*', '-']
energy_after = -700, energy_bofore = -1000

Переход состоялся random_value = 0.8799689681972056 < probably_change = 0.8866730501492976
{0: (1, 1), 1: (2, 2)}
['-', '-', '-']
['-', '*', '-']
['-', '-', '*']
energy_after = -1000, energy_bofore = -700
different_energy = -300 <= 0
{0: (1, 1), 1: (0, 1)}
Переход разрешен 
['-', '-', '-']
['*'

In [32]:
print({1: 2, 3: 4}.values)

dict_values([2, 4])
